# test GUIDER ROI production code, v2
## testing this against what is in the butler
Craig Lage - 20Nov25

In [ ]:
from lsst.daf.butler import Butler
from lsst.ts.observatory.control.utils.extras.guider_roi import GuiderROIs  # noqa: F401                                                                                                                  

from astropy.table import Table, vstack
from matplotlib import pyplot as plt
import numpy as np
from tqdm import tqdm

from lsst.summit.utils.efdUtils import makeEfdClient
import lsst.summit.utils.butlerUtils as butlerUtils
from lsst.obs.lsst import LsstCam
from lsst.summit.utils.butlerUtils import getExpRecordFromDataId
from lsst.afw import cameraGeom

In [ ]:
repo_name = "LSSTCam"
catalog_dataset = "guider_roi_monster_guide_catalog"
vignetting_dataset = "guider_roi_vignetting_correction"
collection = "guider_roi_data"
groi = GuiderROIs(
                catalog_name=catalog_dataset,
                vignetting_dataset=vignetting_dataset,
                collection=collection,
                repo_name=repo_name,
            )

In [ ]:
butler = butlerUtils.makeDefaultButler("LSSTCam")
instrument = 'LSSTCam'
camera = LsstCam.getCamera()
client = makeEfdClient()

In [ ]:
expId = 2025111900195
dataId = {'exposure':expId, 'instrument':'LSSTCam'}
expRecord = getExpRecordFromDataId(butler, dataId)
ra = expRecord.tracking_ra
dec = expRecord.tracking_dec
skyang = expRecord.sky_angle

In [ ]:
roi_spec, selected_stars = groi.get_guider_rois(
            ra=ra,
            dec=dec,
            sky_angle=skyang,  # degrees                                                                                                                                                                  
            roi_size=400,  # pixels (FIXED right now, should make an argument)                                                                                                                            
            roi_time=200,  # ms (max allowed by ROICommon validation: 5-200)                                                                                                                              
            band="y",  # CHANGE to Y by HAND                                                                                                                                                              
            use_guider=True,
            use_science=False,
            use_wavefront=False,
        )

In [ ]:
#has_star = ['R00_SG0', 'R00_SG1', 'R04_SG0', 'R04_SG1', 'R40_SG1', 'R44_SG0', 'R44_SG1', ]
no_star = ['R44_SG1', 'R00_SG0', 'R04_SG1']
segs = ['Segment02', 'Segment06', 'Segment15', 'Segment16', 'Segment07', 'Segment17', 'Segment14']
for expId in [2025112700370]:#[2025112100032, 2025112100033, 2025112100034]:
    dataId = {'exposure':expId, 'instrument':'LSSTCam'}
    expRecord = getExpRecordFromDataId(butler, dataId)
    ra = expRecord.tracking_ra
    dec = expRecord.tracking_dec
    skyang = expRecord.sky_angle
    roi_spec, selected_stars = groi.get_guider_rois(
                ra=ra,
                dec=dec,
                sky_angle=skyang,  # degrees                                                                                                                                                                  
                roi_size=400,  # pixels (FIXED right now, should make an argument)                                                                                                                            
                roi_time=200,  # ms (max allowed by ROICommon validation: 5-200)                                                                                                                              
                band="r",  # CHANGE to Y by HAND                                                                                                                                                              
                use_guider=True,
                use_science=False,
                use_wavefront=False,
            )    
    print(expId)
    print(f"RA={ra}, Dec={dec}, Sky_angle={skyang}")
    print()
    match = 0
    for detector in camera:
        if detector.getType()== cameraGeom.DetectorType.GUIDER :
            detName = detector.getName()
            idet = detector.getId()
            guider_raw = butler.get('guider_raw', dataId=dataId, detector=idet)
            mdata = guider_raw.metadata
            cols = mdata['ROICOLS']
            rows = mdata['ROIROWS']
            llX = mdata['ROICOL']
            llY = mdata['ROIROW']
            seg = mdata['ROISEG']
            print(f"From butler,   {detName} segment={seg.replace('Segment', '')}, startRow={llY}, startCol={llX}")
            print(f"From roi_spec, {detName} {roi_spec.roi[detName.replace('_', '')]}")
            
            for i, ampName in enumerate(selected_stars['ampName']):
                if selected_stars['ccdName'][i] == detName:
                    print(f"From selected_stars, {selected_stars['ccdName'][i]}  {ampName}   startRow={selected_stars['ampyLL'][i]:.1f}  startCol=={selected_stars['ampxLL'][i]:.1f}   mag={selected_stars['gaia_G'][i]:.2f}  ampX={selected_stars['ampX'][i]:.1f}  ampY={selected_stars['ampY'][i]:.1f}  ")
                    if detName in no_star:
                        print("No star in ROI")
                    else:
                        print(f"Star in ROI - {segs[match]}")
                        match += 1

                        
            print()


In [ ]:
selected_stars

In [ ]:
selected_stars.columns


In [ ]:
test = selected_stars['mag_g', 'ccdName', 'ampName']

In [ ]:
print("detName  Amp  gaia_G  \t ampX  ampY  ampxLL  ampyLL")
for i, ampName in enumerate(selected_stars['ampName']):
    print(f"{selected_stars['ccdName'][i]}  {ampName}   {selected_stars['gaia_G'][i]:.2f} \t {selected_stars['ampX'][i]:.1f}  {selected_stars['ampY'][i]:.1f}  {selected_stars['ampxLL'][i]:.1f}  {selected_stars['ampyLL'][i]:.1f}")

In [ ]:
selected_stars.columns

In [ ]:
selected_stars

In [ ]:
expId = 2025112000053
dataId = {'exposure':expId, 'instrument':'LSSTCam'}
expRecord = getExpRecordFromDataId(butler, dataId)

for detector in camera:
        if detector.getType()== cameraGeom.DetectorType.GUIDER :
            detName = detector.getName()
            idet = detector.getId()
            guider_raw = butler.get('guider_raw', dataId=dataId, detector=idet)
            mdata = guider_raw.metadata
            cols = mdata['ROICOLS']
            rows = mdata['ROIROWS']
            llX = mdata['ROICOL']
            llY = mdata['ROIROW']
            seg = mdata['ROISEG']
            ROIsplit = mdata['ROISPLIT']
            print(f"{detName} segment={seg.replace('Segment', '')}, startRow={llY}, startCol={llX}, ROIsplit = {splitROI}")


In [ ]:
mdata['SHUTTER OPEN STARTTIME TAI ISOT']

In [ ]:
guider_raw[20].metadata['STMPTIME']